In [1]:
import pandas as pd
import numpy as np
import json
import re


# ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
imdb_reviews_path = 'data/IMDB Dataset.csv/IMDB Dataset.csv'
twitter_sentiments_path = 'data/trainingandtestdata/training.1600000.processed.noemoticon.csv'
amazon_magazine_subscription_json_path = 'data/Magazine_Subscriptions.json'
amazon_magazine_subscription_path = "data/amazon_magazine_reviews.csv"

In [29]:
# Converting json data to csv and saving it
# amazon_magazine_subscription_json_path = 'data/Magazine_Subscriptions.json'
# amz_data = []
# with open(amazon_magazine_subscription_json_path, 'r') as f:
#     amz_data = f.readlines()
# amz_data = [eval(data.strip().replace("false", "False").replace("true", "True")) for data in amz_data]
# amz_df = pd.DataFrame.from_dict(amz_data)
# amz_df.to_csv("data/amazon_magazine_reviews.csv", index = False)

In [6]:
imdb_df = pd.read_csv(imdb_reviews_path)

In [34]:
imdb_df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [59]:
tweet_df = pd.read_csv(twitter_sentiments_path, encoding = "ISO-8859-1", header = None)
tweet_df.rename(columns = {0:"sentiments",1:"ids",2:"DateTime",3:"NO_QUERY",4:"UserId",5:"review"}, inplace = True)

In [60]:
tweet_df.columns

Index(['sentiments', 'ids', 'DateTime', 'NO_QUERY', 'UserId', 'review'], dtype='object')

In [61]:
amz_df = pd.read_csv(amazon_magazine_subscription_path)
amz_df.rename(columns = {"reviewText":"review"}, inplace = True)

In [62]:
amz_df.columns

Index(['overall', 'vote', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'reviewerName', 'review', 'summary', 'unixReviewTime', 'style',
       'image'],
      dtype='object')

In [63]:
len(imdb_df), len(tweet_df), len(amz_df)

(50000, 1600000, 89689)

In [82]:
imdb3000 = pd.DataFrame(imdb_df['review'][:3000].copy())
tweet3000 = pd.DataFrame(tweet_df['review'][:3000].copy())
amz4000 = pd.DataFrame(amz_df['review'][:4000].copy())

In [215]:
main_df = pd.concat([imdb3000, tweet3000, amz4000]).rename(columns = {'review':'sentence'})

In [216]:
main_df['class'] = [0] * 10000

def parse(sent):
    parsedSentence = ""
    for ch in sent.split(" "):
        if ch == ".":
            break
        parsedSentence += ch
main_df['review'] = main_df['review'].apply(lamda sent: parse(sent))

# Pre processing sentences

### Removal of html tags and encoding characters
    - HTML tags and encoding characters will make no use to the sentences when generating machine text for the 
    given sentences

### Setting a sentence length
    - Setting a hard barrier limit of <= 25 words for a given sentence so we can generate machine generated text on Chatgpt. 
      To form such sentences we will check two conditions 
    - First if the sentence has  <= 25 words we will take the whole sentence
    - Otherwise we will take the sentence ending near 25 words
    - For that we will check for two conditions.
          1.   If the sentence is "I am going to read. I am reading." Here we see that "read." has a period at its end
               We will take the processed sentence till that word.
          2.   If the sentence is "I am going to read .I am reading." Here we see read ends and there is a space 
               and then period with an I as "read .I" So if we have a period at the starting index of the word we will 
               take the previous word and end our processed sentence there and add a space with a period to keep the 
               actual sentence structure
    - In scenarios where we have period in between two words like "Mr.ABC" or "I am reading.I read" we will ignore the 
      period semantics, and check for other words if the have periods
    - If we dont have any of these conditions we will directly grab the first 25 words
  
        

In [217]:
# Compiling re expressions for data cleaning
htmlRE = re.compile('<.*?>')
htmlCharsRE = re.compile('&#\d+;')

# Functions for data cleaning
def processData(review):
    review = str(review)
    review = re.sub(htmlRE, '', review) # removing HTML
    review = re.sub(htmlCharsRE, '', review) # removing HTML encoding charsets
    words = review.split()

    if len(words) > 25:
        idxs = []
        for idx,wrd in enumerate(words):
            if wrd[-1] == ".":
                idxs.append(idx)
            elif wrd[0] == ".":
                idxs.append(idx)


        # getting the closest index
        final_idx_sort = sorted([(abs(i-24),i) for i in idxs])

        if len(final_idx_sort) == 0:
            review = " ".join(words[:25])
        else:
            final_dist, final_idx = final_idx_sort[0]
        

            if final_idx > 24 and final_dist > 6:
                review = " ".join(words[:25])
            else:
                word = words[final_idx]

                if word[-1] == ".":
                    review = " ".join(words[:final_idx+1])
                else:
                    review = " ".join(words[:final_idx])
                    review += " ."

    return review

# apply data cleaning on the dataframe
main_df['sentence'] = main_df['sentence'].apply(lambda x: processData(x))


In [218]:
main_df.to_csv("data/mainDataset10000.csv")

In [200]:
with open("h.txt", "r") as f:
    chatData = f.readlines()

In [202]:
chatData

['(0). According to another reviewer, watching just one episode of Oz is enough to get you hooked.\n',
 '\n',
 '(1). The filming style of this production is unassuming and reminiscent of older BBC programs, which adds to the realism of the piece, sometimes in a discomforting way.\n',
 '\n',
 '(2). Spending a too-hot summer weekend in the air-conditioned theater to watch a light-hearted comedy was a wonderful way to pass the time.\n',
 '\n',
 "(3). The movie is about a family with a little boy named Jake who believes there's a zombie in his closet, and whose parents are constantly fighting. However, the film moves slower than a soap opera.\n",
 '\n',
 '(4). Love in the Time of Money, directed by Petter Mattei, is visually stunning and offers a vivid portrayal of human relationships.\n',
 '\n',
 '(5). This movie is likely my favorite of all time. It tells a story of selflessness, sacrifice, and dedication to a noble cause without being preachy or boring.\n',
 '\n',
 '(6). I would enjoy s

In [205]:
data = []
for line in chatData:
    if line != "\n":
        data.append(line.strip()[5:])
len(data)

10

In [206]:
cols = [""] * 10000

cols[:9] = data

In [207]:
cols[:20]

['According to another reviewer, watching just one episode of Oz is enough to get you hooked.',
 'The filming style of this production is unassuming and reminiscent of older BBC programs, which adds to the realism of the piece, sometimes in a discomforting way.',
 'Spending a too-hot summer weekend in the air-conditioned theater to watch a light-hearted comedy was a wonderful way to pass the time.',
 "The movie is about a family with a little boy named Jake who believes there's a zombie in his closet, and whose parents are constantly fighting. However, the film moves slower than a soap opera.",
 'Love in the Time of Money, directed by Petter Mattei, is visually stunning and offers a vivid portrayal of human relationships.',
 'This movie is likely my favorite of all time. It tells a story of selflessness, sacrifice, and dedication to a noble cause without being preachy or boring.',
 'I would enjoy seeing an updated version of the Seahunt series that incorporates modern technology.',
 "W

In [212]:
chatgpt_df = pd.DataFrame(cols).rename(columns = {0:"sentence"})

In [213]:
chatgpt_df

,sentence
0,"According to another reviewer, watching just o..."
1,The filming style of this production is unassu...
2,Spending a too-hot summer weekend in the air-c...
3,The movie is about a family with a little boy ...
4,"Love in the Time of Money, directed by Petter ..."
...,...
9996,
9997,
9998,
9999,
